In [1]:
pip install black

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     --------- ------------------------------ 0.3/1.2 MB 9.6 MB/s eta 0:00:01
     ---------------------------- ----------- 0.9/1.2 MB 11.0 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 11.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [43]:
# remover .txt e .lab caso o .lab correspondente esteja vazio

import os
import shutil

# Set the source and destination directories
src_dir = 'E:\\Renato\\desktop-20230306\\resumes_corpus'
dest_dir = 'E:\\Renato\\desktop-20230306\\empty'

# Loop through all the files in the source directory
for file in [f for f in os.listdir(src_dir) if f.endswith(".lab")]:
    src_file = os.path.join(src_dir, file)
    dest_file = os.path.join(dest_dir, file)
    file_twin = file.split(".")[0] + ".txt"
    src_file_twin = os.path.join(src_dir, file_twin)
    dest_file_twin = os.path.join(dest_dir, file_twin)

    # Check if the file is empty
    if os.path.getsize(src_file) == 0:
        print(src_file)
        shutil.move(src_file, dest_file)
        shutil.move(src_file_twin, dest_file_twin)

In [3]:
import os
import sys
import pandas as pd
import numpy as np
import spacy
import re
import unidecode
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')

def createDataFrame(path, extension, label, delimiter):
    # Create an empty DataFrame
    df = pd.DataFrame()
    vazios = []

    docs_names = os.listdir(path)    
    txt_files = [d for d in docs_names if d.endswith(extension)]

    with tqdm(total=len(txt_files), file=sys.stdout, colour='red', \
        desc='Loading ') as pbar:

        # Loop through all the files in the directory
        for filename in os.listdir(path):
            if filename.endswith(extension):
                # Read the file into a DataFrame
                file_path = os.path.join(path, filename)
                if delimiter:
                    temp_df = pd.read_csv(file_path, delimiter='\t', encoding = 'ISO-8859-1', header=None)
                else:
                    temp_df = pd.read_csv(file_path, encoding = 'ISO-8859-1', header=None)
                # Add a new column with the filename
                temp_df['filename'] = filename.split(sep='.')[0]
            
                # Append the DataFrame to the main DataFrame
                df = pd.concat([df, temp_df])
                pbar.update(1)

    # Reset the index to start from 0
    df = df.reset_index(drop=True)
    df = df.rename(columns={0:label})
    df['filename']=df['filename'].astype(str)
    print(f"Vazios:  {len(vazios)}")
    return df

def preprocessing(txt):
    texts_pre = []
    texts_preprocessed = []
    spacy_stopwords = nlp.Defaults.stop_words
    stemmer = PorterStemmer()
    # convert all characters in the string to lower case
    txt = txt.lower()
    # remove non-english characters, punctuation and numbers
    txt = re.sub(r'<.*?>', '', txt)
    txt = re.sub('[^a-zA-Z0-9]', ' ', txt) #
    txt = re.sub('http\S+\s*', ' ', txt)  # remove URLs
    txt = re.sub('RT|cc', ' ', txt)  # remove RT and cc
    txt = re.sub('#\S+', '', txt)  # remove hashtags
    txt = re.sub('@\S+', '  ', txt)  # remove mentions
    txt = re.sub('\s+', ' ', txt)  # remove extra whitespace
    txt = txt.replace('\n', ' ')
    txt = unidecode.unidecode(txt)
    doc = nlp(txt)
    tokens = [t.text.lower() for t in doc]
    text = ' '.join(tokens) 
    text_without_stopword = [word for word in text.split() if word not in spacy_stopwords]
    txt_final = [stemmer.stem(w) for w in text_without_stopword]
    return txt_final

def sampling(df3, x):
    
    # shuffle a dataframe
    df3 = df3.sample(frac=1, random_state=42)

    # get unique values of column A
    unique_values = df3['label'].unique()

    # create an empty dataframe to store the results
    result_df = pd.DataFrame(columns=df3.columns)

    # loop through the unique values and append 500 rows to the result dataframe for each value
    for value in unique_values:
        temp_df = df3[df3['label'] == value].iloc[:x]
        result_df = result_df.append(temp_df, ignore_index=True)
    
    return result_df

def vectorizer(df):
    # vectorize text data
    vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=None)
    X_vect = vectorizer.fit_transform(df['Resume_nlp']).toarray()
    # countvectorizer_test = vectorizer.transform(X_test).astype(float)

    new_df = pd.DataFrame(X_vect, columns=vectorizer.get_feature_names_out())
    return new_df

def main():
    path = 'E:\\Renato\\Mestrado\\dissertacao_v2\\resumes_corpus'
    extension = ".txt"
    label = 'resume'
    extension_2 = ".lab"
    label_2 = 'label'
    df = createDataFrame(path, extension, label, delimiter=True)
    df2 = createDataFrame(path, extension_2, label_2, delimiter=False)
    df2 = df2.groupby('filename').agg(lambda x: ' '.join(map(str, x))).reset_index()
    df2['label'] = df2['label'].str.split().str[0]
    df3 = df.merge(df2, on='filename', how='inner')
    result_df = sampling(df3, 50)
    result_df['Resume_nlp'] = result_df['resume'].apply(lambda w: preprocessing(w)).astype(str)
    new_df = vectorizer(result_df)

if __name__ == '__main__':
    main()

Loading : 100%|██████████| 29035/29035 [08:11<00:00, 59.13it/s]
Vazios:  0
Loading : 100%|██████████| 29035/29035 [01:53<00:00, 256.13it/s]
Vazios:  0


C:\Users\Renato\AppData\Local\Temp\ipykernel_11652\729890280.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)


In [4]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29035 entries, 0 to 29034
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   resume    29035 non-null  object
 1   filename  29035 non-null  object
 2   label     29035 non-null  object
dtypes: object(3)
memory usage: 907.3+ KB


In [5]:
df3.head()

,resume,filename,label
0,"Database Administrator <span class=""hl"">Databa...",00001,Database_Administrator
1,"Database Administrator <span class=""hl"">Databa...",00002,Database_Administrator
2,Oracle Database Administrator Oracle <span cla...,00003,Database_Administrator
3,Amazon Redshift Administrator and ETL Develope...,00004,Database_Administrator
4,Scrum Master Scrum Master Scrum Master Richmon...,00005,Database_Administrator


In [6]:
df3['label'].value_counts()

Software_Developer        5828
Systems_Administrator     4182
Project_manager           3527
Web_Developer             3466
Database_Administrator    2784
Java_Developer            2418
Python_Developer          2311
Network_Administrator     2260
Security_Analyst          2259
Name: label, dtype: int64

In [68]:
# result_df = sampling(df3, 50)

C:\Users\Renato\AppData\Local\Temp\ipykernel_8968\1162965336.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)


In [69]:
result_df['label'].value_counts()

Database_Administrator    500
Systems_Administrator     500
Project_manager           500
Software_Developer        500
Web_Developer             500
Security_Analyst          500
Network_Administrator     500
Java_Developer            500
Python_Developer          500
Name: label, dtype: int64

In [8]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   resume    450 non-null    object
 1   filename  450 non-null    object
 2   label     450 non-null    object
dtypes: object(3)
memory usage: 10.7+ KB


In [9]:
result_df['Resume_nlp'] = result_df['resume'].apply(lambda w: preprocessing(w)).astype(str)

In [10]:

new_df = vectorizer(result_df)

In [25]:
use_scaler = True
use_gridsearch = True

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

y = result_df['label'].to_numpy()
X = new_df.to_numpy()

classifiers = {
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "GaussianNB": GaussianNB(),
    "SVC": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier(random_state=42),
    "MLP": MLPClassifier(),
}

list_hyperparameters = {
    "KNN": {
        "n_neighbors": [3, 5, 7, 11, 13],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan"],
    },
    "GaussianNB": {"var_smoothing": np.logspace(0, -9, num=100)},
    "Decision Tree": {
        "max_leaf_nodes": list(range(2, 100)),
        "min_samples_split": [2, 3, 4],
    },
    "SVC": [
        {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1.0, 10, 100, 1000]},
        {"kernel": ["linear"], "C": [1.0, 10, 100, 1000]},
    ],

    "RandomForest": {
        "n_estimators": [200, 500],
        "max_features": ["auto", "sqrt", "log2"],
        "max_depth": [4, 5, 6, 7, 8],
        "criterion": ["gini", "entropy"],
    },

    "MLP": {
        "hidden_layer_sizes": [(3, 30, 10), (20,)],
        "activation": ["relu", "tanh"],
        "solver": ["sgd", "adam"],
        "alpha": [0.0001, 0.05],
        "learning_rate": ["constant", "adaptive"],
    },
}


scaler = MinMaxScaler()

for clf_name, clf in classifiers.items():
    print("\nClassifier:", clf_name)

    results_accuracy = []
    results_precision = []
    results_recall = []
    results_f1_score = []

    hyperparameters = None

    if use_gridsearch:
        hyperparameters = list_hyperparameters[clf_name]
        clf = GridSearchCV(clf, hyperparameters, n_jobs=-1)

    print("\n  Cross Validation\n")

    for k, (ids_train, ids_test) in enumerate(skfold.split(X, y)):
        X_train = X[ids_train]
        y_train = y[ids_train]

        X_test = X[ids_test]
        y_test = y[ids_test]

        if use_scaler:
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        accuracy_fold = accuracy_score(y_test, y_pred, normalize=False)
        precision_fold = precision_score(y_test, y_pred, average="macro")
        recall_fold = recall_score(y_test, y_pred, average="macro")
        f1_fold = f1_score(y_test, y_pred, average="macro")

        results_accuracy.append(accuracy_fold)
        results_precision.append(precision_fold)
        results_recall.append(recall_fold)
        results_f1_score.append(f1_fold)

        print(
            "    K:",
            k + 1,
            "-",
            len(X_train),
            "-",
            len(X_test),
            "-",
            precision_fold,
            "-",
            recall_fold,
            "-",
            f1_fold,
        )

        if use_gridsearch:
            print("    Best parameters:", clf.best_params_)

    print(
        "\n  Mean Accuracy:",
        np.mean(results_accuracy),
        "-",
        np.std(results_accuracy),
    )
    print(
        "\n  Mean Precision:",
        np.mean(results_precision),
        "-",
        np.std(results_precision),
    )
    print("  Mean Recall:", np.mean(results_recall), "-", np.std(results_recall))
    print("  Mean F1 Score:", np.mean(results_f1_score), "-", np.std(results_f1_score))



Classifier: KNN

  Cross Validation

    K: 1 - 360 - 90 - 0.5792592592592594 - 0.4444444444444444 - 0.43494501141559966
    Best parameters: {'metric': 'euclidean', 'n_neighbors': 13, 'weights': 'distance'}


C:\Users\Renato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


    K: 2 - 360 - 90 - 0.6045159194282002 - 0.39999999999999997 - 0.37342376652721476
    Best parameters: {'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'}


C:\Users\Renato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


    K: 3 - 360 - 90 - 0.4164292497625831 - 0.30000000000000004 - 0.23533724340175954
    Best parameters: {'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'distance'}
    K: 4 - 360 - 90 - 0.6867224367224368 - 0.37777777777777777 - 0.36418279722409885
    Best parameters: {'metric': 'euclidean', 'n_neighbors': 13, 'weights': 'uniform'}


...

C:\Users\Renato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


    K: 3 - 360 - 90 - 0.6294091710758377 - 0.611111111111111 - 0.6068095083832958
    Best parameters: {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'solver': 'adam'}


C:\Users\Renato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


    K: 4 - 360 - 90 - 0.739568618515987 - 0.6444444444444444 - 0.6442996414598848
    Best parameters: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'solver': 'sgd'}
    K: 5 - 360 - 90 - 0.6621659204992538 - 0.6444444444444444 - 0.646997532495805
    Best parameters: {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (20,), 'learning_rate': 'constant', 'solver': 'adam'}

  Mean Accuracy: 55.2 - 2.481934729198171

  Mean Precision: 0.6418897476792214 - 0.05598755681647637
  Mean Recall: 0.6133333333333333 - 0.02757705254664633
  Mean F1 Score: 0.6101439977802013 - 0.03110533974934251


C:\Users\Renato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [12]:
new_df.head()

,00,000,000webhostapp,01,0172,02,02110,029736ce7470,03,04,...,zoom,zope,zrtool,zs,zt,zuora,zurich,zuul,zuzyaepn7kcmjacv,zxing
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038694
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


<h1>PERA LA</h1>

In [15]:
# # vectorize text data
# vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=None)
# X_vect = vectorizer.fit_transform(result_df['Resume_nlp']).toarray()
# # countvectorizer_test = vectorizer.transform(X_test).astype(float)

# new_df = pd.DataFrame(X_vect, columns=vectorizer.get_feature_names_out())

y = result_df['label']
X = new_df

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

"""# **Random Forest Classifier**"""

RF_Model = RandomForestClassifier(random_state=42, n_estimators= 500, max_depth=8, criterion='gini')
RF_Model.fit(X_train, Y_train)

prediction=RF_Model.predict(X_test)
prediction

print("training Score: {:.2f}".format(RF_Model.score(X_train, Y_train)))
print("test Score: {:.2f}".format(RF_Model.score(X_test, Y_test)))

print("model report: %s: \n %s\n" % (RF_Model, metrics.classification_report(Y_test, prediction)))

training Score: 1.00
test Score: 0.67
model report: RandomForestClassifier(max_depth=8, n_estimators=500, random_state=42): 
                         precision    recall  f1-score   support

Database_Administrator       0.91      0.83      0.87        12
        Java_Developer       0.81      0.93      0.87        14
 Network_Administrator       0.40      1.00      0.57         8
       Project_manager       0.64      0.78      0.70         9
      Python_Developer       1.00      0.88      0.93         8
      Security_Analyst       1.00      0.86      0.92         7
    Software_Developer       0.50      0.17      0.25        12
 Systems_Administrator       0.80      0.29      0.42        14
         Web_Developer       0.30      0.50      0.37         6

              accuracy                           0.67        90
             macro avg       0.71      0.69      0.66        90
          weighted avg       0.72      0.67      0.65        90




In [24]:
bbbb = y.to_numpy()
bbbb

array(['Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 'Software_Developer',
       'Software_Developer', 'Software_Developer', 

In [20]:
X[]

KeyError: 0